# Lesson 1 - Source code overview and compilation 

## Overview
In this lesson, we cover the basic structure of the WRF-Hydro source code, discuss the various compile-time options, and compile the model. 

### Software
Current, stable WRF-Hydro source code can be obtained from the WRF-Hydro website at https://ral.ucar.edu/projects/wrf_hydro/model-code. 

The full repository including bleeding-edge versions can be obtained from GitHub at https://github.com/NCAR/wrf_hydro_nwm_public. 

For a detailed description of model see the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide).

## Orientation to the WRF-Hydro source code directory 

### Directory structure
The top-level directory structure of the code is described below as nested under *trunk/NDHMS*. The table below provides a brief descriptions of the file contents of each subdirectory.  

**Table 1.** Descriptions of the contents of each source code sub-directory.

| File/directory name | Description |
| ------------- |:-------------|
| arc/ | Contains macro files, which specify the compile configurations, compiler options, links to netCDF libraries, etc. |
| cmake_modules/ | Utilities for the experimental CMake build |
| CPL/Noah_cpl/ | WRF-Hydro coupling interface for coupling WRF-Hydro components with the standalone (offline) Noah land surface model data assimilation and forecasting system |
| CPL/NoahMP_cpl/ | WRF-Hydro coupling interface for coupling WRF-Hydro components with the standalone (offline) Noah-MP land surface model data assimilation and forecasting system |
| CPL/WRF_cpl/ | WRF-Hydro coupling interface for coupling WRF-Hydro components with the WRF system |
| CPL/CLM_cpl/, CPL/LIS_cpl/, CPL/NUOPC_cpl/ | Work in progress for ongoing coupling work |
| Data_Rec/ | Contains some data declaration modules |
| Debug_Utilities/ | Debugging utilities |
| deprecated/ | Files not currently in use |
| Doc/ | Pointer to location of full documentation |
| HYDRO_drv/ | High-level WRF-Hydro component driver |
| IO/ | I/O interfaces |
| Land_models/Noah/ | Noah land surface model driver for standalone applications |
| Land_models/NoahMP/ | Noah-MP land surface model driver for standalone applications |
| MPP/ | MPI parallelization routines and functions |
| nudging/ | Nudging data assimilation routines and functions |
| OrchestratorLayer/ | Modules for namelist reads and eventually high level model orchestration |
| Rapid_routing/ | Contains code for the RAPID routing model coupling (unsupported and out of date) |
| Routing/ | Modules and drivers related to specific routing processes in WRF-Hydro |
| template/ | Example namelist files for Noah, Noah-MP, and the WRF-Hydro modules (HYDRO) and example parameter tables for HYDRO. **Note:** Parameter tables for Noah and Noah-MP are stored within the Land_models directory. A sample bash script (setEnvar.sh) that could be passed to the compile script listing compile time options for WRF-Hydro is also located here. |
| utils/ | Generic utilities used throughout the code |
| compile_offline_Noah.sh | Script for compiling WRF-Hydro standalone version with the Noah land surface model |
| compile_offline_NoahMP.sh | Script for compiling WRF-Hydro standalone version with the Noah-MP land surface model |
| configure | Script to configure the WRF-Hydro compilation |
| Makefile | Top-level makefile for building and cleaning WRF-Hydro code |
| README.build.txt | WRF-Hydro build instructions for the standalone model |
| wrf_hydro_config | Configure script for coupled WRF-Hydro configuration |
| *.json | JSON files used for testing |

See the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide) for more detailed information on individual Fortran modules.

## Compiling WRF-Hydro

### WRF-Hydro compile-time options
Compile-time options are choices about the model structure that are determined when the model is compiled. Table 2 below provides a description of the compile time options.

**Table 2.** Description of WRF-Hydro compile time options.

| Variable | Options | Description |
|-------------|-------------|:-------------|
| WRF_HYDRO | 1=On | Always set to 1 for compiling WRF-Hydro. |
| HYDRO_D | 0=Off, 1=On | Enhanced diagnostic output for debugging. |
| SPATIAL_SOIL | 0=Off, 1=On | Spatially distributed parameters for Noah-MP. This allows Noah-MP to use spatially distributed parameters for the land surface model rather than parameter based upon soil class and land use category look up tables. See the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide) for more information. |
| WRF_HYDRO_RAPID | 0=Off, 1=On | Coupling with the RAPID routing model. This option is not currently supported.  |
| NCEP_WCOSS | 0=Off, 1=On | WCOSS file units. Do not use unless working on the WCOSS machines. |
| NWM_META | 0=Off, 1=On | NWM output metadata. Do not use unless running the operational NWM. |
| WRF_HYDRO_NUDGING | 0=Off, 1=On | Streamflow nudging. Enable the streamflow nudging routines for Muskingum-Cunge Routing. See the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide) for more information. |

### Compiling WRF-Hydro in uncoupled mode
In this section we compile the model in uncoupled or standalone mode using the Noah-MP land-surface model. 

The WRF-Hydro source code currently supports compilation with Intel and GNU compilers. See the [How To Build & Run WRF-Hydro in Standalone Mode user guide ](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide) for information on system requirements.  

**Step 1. List the contents of the *trunk/NDHMS* directory.**

In [ ]:
ls ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS

**Step 2. Configure the compilation environment.**

As noted in Table 1, the script `configure` is used to configure the compilation environment. Execute the `configure` script and you will be prompted with a choice of compiler. Because we are using the GNU Fortran compiler in our training environment, we will select *option 2* for GNU / gfortran.

**NOTE: If you are running this tutorial on your own system, select the appropriate compiler for your system.**

In [ ]:
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/
./configure 2

**Step 3. Make a copy of the template environment variable file, *template/setEnvar.sh***

As described in Table 1, the `template/setEnvar.sh` script can be used to specify compile-time options by setting environment variables. We will use this method of compilation for this lesson.

In [ ]:
cp template/setEnvar.sh setEnvar.sh

**Step 4. Set compile-time options**

Edit the `~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh` script to set environment variables required by the compile script. Your `setEnvar.sh` should look like the bash script below when you are finished.  

---

```bash
#!/bin/bash
  
# WRF-Hydro compile time options

# This is a WRF environment variable. Always set to 1=On for compiling WRF-Hydro.
export WRF_HYDRO=1

# Enhanced diagnostic output for debugging: 0=Off, 1=On.
export HYDRO_D=1

# Spatially distributed parameters for NoahMP: 0=Off, 1=On.
export SPATIAL_SOIL=1

# RAPID model: 0=Off, 1=On.
export WRF_HYDRO_RAPID=0

# WCOSS file units: 0=Off, 1=On.
export NCEP_WCOSS=0

# NWM output metadata: 0=Off, 1=On.
export NWM_META=0

# Streamflow nudging: 0=Off, 1=On.
export WRF_HYDRO_NUDGING=0
```

---


**Step 5. Compile WRF-Hydro in standalone mode**

We support compiling the standalone WRF-Hydro model with either the Noah or Noah-MP land surface models. Two helper scripts are supplied, `compile_offline_Noah.sh` and `compile_offline_NoahMP.sh`, to assist users with the compilation process. 

We will be compiling with the Noah-MP land surface model for this lesson and thus we will be using the `compile_offline_NoahMP.sh` script.

The `compile_offline_Noah.sh` and `compile_offline_NoahMP.sh` execute a similar process during compilation, and this process is described in more detail in [How to Build & Run WRF-Hydro in Standalone Mode](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide) and the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide). One action that these scripts perform is to source the `setEnvar.sh` script to specify compile-time options by setting environment variables.

We will now compile the model by executing the `compile_offline_NoahMP.sh` script and supplying our `setEnvar.sh` script as the first argument. Additionally, we will pipe the output from the compilation process to a log file because compilation can generate a lot of output.

In [ ]:
./compile_offline_NoahMP.sh setEnvar.sh >> compile.log 2>&1

**Step 6. Check the compile log to make sure that compilation completed successfully**

The last few lines of your compile log should indicate that "Make was successful" and the environment variables used in the compile.

In [ ]:
tail -13 compile.log

## Exploring compilation artifacts
After a successful compilation, there will be a new directory created called `Run` in the `trunk/NDHMS` directory. The `Run` directory contains the compiled binary `wrf_hydro.exe` as well as a number of template input files to assist users with parameterizing WRF-Hydro.

**Check the contents of the Run directory**

In [ ]:
ls Run

The Run directory now contains parameter tables, two namelist files, and the executable. 

Table 3 below briefly describes the contents of the Run directory and more information on the individual files can be found in the [Technical Description](https://ral.ucar.edu/projects/wrf_hydro/technical-description-user-guide).

**Table 3.** Description of the file contents of the `Run` directory.

| Filename | Description |
| ------------- |:-------------|
| CHANPARM.TBL| Channel routing parameter table. |
| GENPARM.TBL | This file contains global parameters for the Noah-MP land surface model. |
| HYDRO.TBL | Parameter table for lateral flow routing within WRF-Hydro. In the HYDRO.TBL file parameters are specified by land cover type or soil category. |
| MPTABLE.TBL | Land surface model parameters that are a function of land cover type. |
| SOILPARM.TBL | Land surface model parameters assigned based upon the soil classification. |
| hydro.namelist | Specifies the settings for all of the routing components of WRF-Hydro. |
| namelist.hrldas | Specifies the land surface model options to be used. |
| wrf_hydro.exe | WRF-Hydro executable/binary file. |
| wrf_hydro_NoahMP.exe | Symbolic link to the executable file compiled for WRF-Hydro with Noah-MP. |

# Next up - Running WRF-Hydro
This concludes Lesson 1. In the [next lesson](Lesson-2-run.ipynb) we will run a basic WRF-Hydro simulation using a prepared domain and briefly discuss run-time options.

**IT IS BEST TO EITHER SHUTDOWN THIS LESSON OR CLOSE IT BEFORE PROCEEDING TO THE NEXT LESSON TO AVOID POSSIBLY EXCEEDING ALLOCATED MEMORY. Shutdown the lesson be either closing the browser tab for the lesson or selecting `Kernel -> Shut Down Kernel` in JupyterLab.**

© UCAR 2020